In [ ]:
from os.path import join
import os
import pickle

import numpy as np
import matplotlib.pyplot as plt
import torch

from src.model.get_model import get_multistep_linear_model
from src.env.MiniFurnace import MiniFurnace
from src.control.mpc import MPC
from src.utils.data_preprocessing import get_data

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# MPC with trained model

## MPC Hyperparameters

In [ ]:
env = MiniFurnace()
observation_min = env.observation_space['low']
observation_max = env.observation_space['high']
action_min = env.action_space['low']
action_max = env.action_space['high']

model_name = 'multistep_linear_res2'

# Set your best model hyperparameter
state_order = 1
action_order = 1

receding_horizon = 7
max_iter = 400
is_logging = True
opt_config = {'lr': 1e-2}

## Prepare for MPC

In [ ]:
state_dim = env.state_dim
action_dim = env.action_dim

saved_model_path = 'saved_model/{}/best_model_{}_{}.pt'.format(model_name, state_order, action_order)
m = get_multistep_linear_model(model_name, state_dim, action_dim, state_order, action_order, saved_model_path=saved_model_path).to(device)
m.eval()
mpc_solver = MPC(model=m,
                 action_dim=action_dim,
                 receding_horizon=receding_horizon,
                 action_min=action_min,
                 action_max=action_max,
                 gamma=env.gamma,
                 max_iter=max_iter,
                 is_logging=is_logging,
                 device=device,
                 opt_config=opt_config)

## load reference trajectory

In [ ]:
y = np.transpose(np.load('data/reference_trajectory.npy'))
T = y.shape[0]  # Reference trajectory length

In [ ]:
y.shape

## Run MPC

In [ ]:
observation_trajectory = []
action_trajectory = []
log_trajectory = []

past_observations, past_actions = env.reset()
past_observations, past_actions = past_observations[-state_order:, :], past_actions[-action_order: , :][1:]

for t in range(T - receding_horizon):
    print('Now [{}] / [{}]'.format(t, T - receding_horizon))
    optimal_actions, log = mpc_solver.solve(x0=past_observations, u0=past_actions, y=y[t:t + receding_horizon, :])
    action = optimal_actions[0, :].reshape(1, -1)  # proceed first action
    observation = env.step(action)

    observation_trajectory.append(observation)
    action_trajectory.append(action)
    log_trajectory.append(log)

    past_observations = np.concatenate((past_observations, observation), axis=0)[1:]
    past_actions = np.concatenate((past_actions, action), axis=0)[1:]

observation_trajectory = np.concatenate(observation_trajectory, axis=0)
action_trajectory = np.concatenate(action_trajectory, axis=0)

mpc_result = {
    'traj_obs': observation_trajectory,
    'traj_action': action_trajectory,
    'log': log_trajectory
}
with open('mpc_result/result.log', 'wb') as f:
    pickle.dump(mpc_result, f)


In [ ]:
for i in range(len(log_trajectory)):
    plt.plot(log_trajectory[i]['loss'])
    plt.yscale('log')

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(5, 25))
axes_flatten = axes.flatten()
for i in range(len(axes_flatten)):
    axes_flatten[i].plot(observation_trajectory[:, i], label='controlled')
    axes_flatten[i].plot(y[:-receding_horizon, i], label='objective')
    axes_flatten[i].legend()
    
    axes_flatten[i].set_ylim([-0.3, 0.3])
    axes_flatten[i].set_title('Observation {}th'.format(i+1))

fig, axes = plt.subplots(3, 1, figsize=(5, 15))
axes_flatten = axes.flatten()
for i in range(len(axes_flatten)):
    axes_flatten[i].plot(action_trajectory[:, i])
    axes_flatten[i].set_ylim([-1.3, 1.3])
    axes_flatten[i].set_title('action {}th'.format(i+1))

# MPC with real model

In [ ]:
initial_obs, initial_actions = env.reset()

In [ ]:
initial_obs, initial_actions = env.reset()
fake_action = np.ones((1, 3))
env_state = env.step(fake_action)

## Copy env's parameter

In [ ]:
tensor_A = torch.from_numpy(env.A).float()  # state_order * state_dim * state_dim 
tensor_B = torch.from_numpy(env.B).float()  # action_order * action_dim * state_dim
tensor_C = torch.from_numpy(env.C).float()  # state_dim

m_real = get_multistep_linear_model(model_name, state_dim, action_dim, state_order, action_order).to(device)
m_real.A.data = tensor_A
m_real.B.data = tensor_B
m_real.C.data = tensor_C.view(1, -1)

In [ ]:
# check model's parameter
model_state = m_real.multi_step_prediction(x0 = torch.zeros(state_order, state_dim), u0=torch.zeros(action_order-1, action_dim), us=torch.ones(1, action_dim))
print(env_state, model_state)

In [ ]:
m_real.eval()
mpc_solver_real = MPC(model=m_real,
                 action_dim=action_dim,
                 receding_horizon=receding_horizon,
                 action_min=action_min,
                 action_max=action_max,
                 gamma=env.gamma,
                 max_iter=max_iter,
                 is_logging=is_logging,
                 device=device,
                 opt_config=opt_config)

In [ ]:
observation_trajectory = []
action_trajectory = []
log_trajectory = []

past_observations, past_actions = env.reset()
past_observations, past_actions = past_observations[-state_order:, :], past_actions[-action_order: , :][1:]

for t in range(T - receding_horizon):
    print('Now [{}] / [{}]'.format(t, T - receding_horizon))
    optimal_actions, log = mpc_solver.solve(x0=past_observations, u0=past_actions, y=y[t:t + receding_horizon, :])
    action = optimal_actions[0, :].reshape(1, -1)  # * (action_max - action_min) + action_min
    observation = env.step(action)

    observation_trajectory.append(observation)
    action_trajectory.append(action)
    log_trajectory.append(log)

    past_observations = np.concatenate((past_observations, observation), axis=0)[1:]
    past_actions = np.concatenate((past_actions, action), axis=0)[1:]

observation_trajectory = np.concatenate(observation_trajectory, axis=0)
action_trajectory = np.concatenate(action_trajectory, axis=0)

mpc_result = {
    'traj_obs': observation_trajectory,
    'traj_action': action_trajectory,
    'log': log_trajectory
}
with open('mpc_result/result.log', 'wb') as f:
    pickle.dump(mpc_result, f)


In [ ]:
for i in range(len(log_trajectory)):
    plt.plot(log_trajectory[i]['loss'])
    plt.yscale('log')

In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(5, 25))
axes_flatten = axes.flatten()
for i in range(len(axes_flatten)):
    axes_flatten[i].plot(observation_trajectory[:, i], label='controlled')
    axes_flatten[i].plot(y[:-receding_horizon, i], label='objective')
    axes_flatten[i].legend()
    
    axes_flatten[i].set_ylim([-0.3, 0.3])
    axes_flatten[i].set_title('Observation {}th'.format(i+1))

fig, axes = plt.subplots(3, 1, figsize=(5, 15))
axes_flatten = axes.flatten()
for i in range(len(axes_flatten)):
    axes_flatten[i].plot(action_trajectory[:, i])
    axes_flatten[i].set_ylim([-1.3, 1.3])
    axes_flatten[i].set_title('action {}th'.format(i+1))